In [88]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
##
import sys
sys.path.append("../")

import gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql import InfluenceRanker, SelfLossInfluenceRanker, AutoFixer, ModelManager, LossRanker, TiresiasRanker, multi_ambiguity_count
from mlsql.models import SimpleCNN, LogReg
from mlsql.utils import setdiff1d
from processors.adultNoCorr import AdultNoCorrProcessor
from processors.mnistbinary import MnistBinaryProcessor
from processors.bhprice import BostonHousingPriceProcessor
from processors.iris import IrisProcessor
from processors.breastCancer import BreastCancerProcessor
from processors.diabetes import DiabetesProcessor
from itertools import groupby
from functools import partial

import logging
import altair as alt
alt.data_transformers.disable_max_rows()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DataTransformerRegistry.enable('default')

In [135]:
seed = 2987429
proc = AdultNoCorrProcessor(seed)
print(proc.ytrain.shape)

(26048,)


In [91]:
def sgd_fn(model, A_data, B_data, labels, last_loss, opt, tol=1e-5):
    with tf.GradientTape() as tape:
        current_loss = model.loss(A_data, B_data, labels)
#     print(current_loss)
    grads = tape.gradient(current_loss, model.variables)
    opt.apply_gradients(zip(grads, model.variables))
    cont = tf.math.abs(last_loss - current_loss) > tol
    return current_loss, cont

def fit(model, A_data, B_data, labels, max_iter=1000):
    opt = tf.keras.optimizers.SGD(learning_rate=0.1)
    last_loss = -np.inf
    for iteration in range(max_iter):
        last_loss, cont = sgd_fn(model, A_data, B_data, labels, last_loss, opt)
        if not cont:
            break
    print(last_loss)        
    print(iteration)
            
def predict(model, A_data, B_data):
    return tf.cast(tf.squeeze(model(A_data, B_data) > 0.5), tf.int32)

In [86]:
model = LogReg(proc, 1)
fit(model, proc.Xtrain, proc.ytrain)

tf.Tensor(0.38511646, shape=(), dtype=float32)
964


In [126]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, proc.Xtrain).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, proc.Xtest).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.85      0.93      0.88     19740
         1.0       0.67      0.48      0.56      6308

    accuracy                           0.82     26048
   macro avg       0.76      0.70      0.72     26048
weighted avg       0.81      0.82      0.81     26048

On Testing
               precision    recall  f1-score   support

         0.0       0.85      0.92      0.89      2518
         1.0       0.63      0.46      0.53       738

    accuracy                           0.82      3256
   macro avg       0.74      0.69      0.71      3256
weighted avg       0.80      0.82      0.80      3256



In [127]:
# split dataset
proc.Xtrain.shape

TensorShape([26048, 17])

In [128]:
A_feature = 9
B_feature = 8
A_data = proc.Xtrain[:, :9]
B_data = proc.Xtrain[:, 9:]
A_test = proc.Xtest[:, :9]
B_test = proc.Xtest[:, 9:]

In [97]:
def grad(model, A_data, B_data, labels):
    var = model.variables
    f1 = model.f1(A_data) # vector m*1
    f2 = model.f2(B_data) # vector m*1
    c1 = var[4]
    c2 = var[5]
    diff = c1*f1+c2*f2-tf.expand_dims(labels, axis=1)
    f1_bias = diff*f1*(1-f1) # vector m*1
    f1_kernel = tf.reduce_mean(f1_bias*A_data, axis=0) # vector m*1 elem-wise multiply matrix m*9, 
                                                       # and reduce the m dimesion
    f1_bias = tf.reduce_mean(f1_bias, axis=0) # vector 1
    f1_kernel = tf.expand_dims(f1_kernel, axis=1)
    
    f2_bias = diff*f2*(1-f2)
    f2_kernel = tf.reduce_mean(f2_bias*B_data, axis=0)
    f2_bias = tf.reduce_mean(f2_bias, axis=0) # vector 1
    f2_kernel = tf.expand_dims(f2_kernel, axis=1)
    
    c1_up = tf.reduce_mean(diff*f1)
    c2_up = tf.reduce_mean(diff*f2)
#     return [f1_kernel, f1_bias, f2_kernel, f2_bias, 0, 0]
    return [f1_kernel, f1_bias, f2_kernel, f2_bias, c1_up, c2_up]

def new_sgd_fn(model, A_data, B_data, labels, last_loss, opt, tol=1e-6):
    current_loss = model.loss(A_data, B_data, labels)
    grads = grad(model, A_data, B_data, labels)
#     print(current_loss)
    opt.apply_gradients(zip(grads, model.variables))
    cont = tf.math.abs(last_loss - current_loss) > tol
    return current_loss, cont

def new_fit(model, A_data, B_data, labels, max_iter=2000):
    opt = tf.keras.optimizers.SGD(learning_rate=0.1)
    last_loss = -np.inf
    for iteration in range(max_iter):
        last_loss, cont = new_sgd_fn(model, A_data, B_data, labels, last_loss, opt)
        if not cont:
            break
    print(last_loss)
    print(iteration)
            
def new_predict(model, A_data, B_data, ratio):
    return tf.cast(tf.squeeze(model(A_data, B_data) > ratio), tf.int32)

In [137]:
from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

tf.Tensor(0.16887231, shape=(), dtype=float32)
tf.Tensor(0.16075721, shape=(), dtype=float32)
tf.Tensor(0.15361165, shape=(), dtype=float32)
tf.Tensor(0.14730522, shape=(), dtype=float32)
tf.Tensor(0.14172716, shape=(), dtype=float32)
tf.Tensor(0.13678333, shape=(), dtype=float32)
tf.Tensor(0.13239262, shape=(), dtype=float32)
tf.Tensor(0.12848607, shape=(), dtype=float32)
tf.Tensor(0.12500383, shape=(), dtype=float32)
tf.Tensor(0.121894725, shape=(), dtype=float32)
tf.Tensor(0.11911404, shape=(), dtype=float32)
tf.Tensor(0.1166233, shape=(), dtype=float32)
tf.Tensor(0.114388935, shape=(), dtype=float32)
tf.Tensor(0.112381466, shape=(), dtype=float32)
tf.Tensor(0.110575624, shape=(), dtype=float32)
tf.Tensor(0.10894877, shape=(), dtype=float32)
tf.Tensor(0.10748134, shape=(), dtype=float32)
tf.Tensor(0.10615609, shape=(), dtype=float32)
tf.Tensor(0.104957774, shape=(), dtype=float32)
tf.Tensor(0.10387286, shape=(), dtype=float32)
tf.Tensor(0.10288962, shape=(), dtype=float32)
tf.Tensor

tf.Tensor(0.08902818, shape=(), dtype=float32)
tf.Tensor(0.089004695, shape=(), dtype=float32)
tf.Tensor(0.08898119, shape=(), dtype=float32)
tf.Tensor(0.0889577, shape=(), dtype=float32)
tf.Tensor(0.08893424, shape=(), dtype=float32)
tf.Tensor(0.08891078, shape=(), dtype=float32)
tf.Tensor(0.08888732, shape=(), dtype=float32)
tf.Tensor(0.08886389, shape=(), dtype=float32)
tf.Tensor(0.088840455, shape=(), dtype=float32)
tf.Tensor(0.08881701, shape=(), dtype=float32)
tf.Tensor(0.08879351, shape=(), dtype=float32)
tf.Tensor(0.08877016, shape=(), dtype=float32)
tf.Tensor(0.08874669, shape=(), dtype=float32)
tf.Tensor(0.088723324, shape=(), dtype=float32)
tf.Tensor(0.08869988, shape=(), dtype=float32)
tf.Tensor(0.08867641, shape=(), dtype=float32)
tf.Tensor(0.08865302, shape=(), dtype=float32)
tf.Tensor(0.08862959, shape=(), dtype=float32)
tf.Tensor(0.088606216, shape=(), dtype=float32)
tf.Tensor(0.088582784, shape=(), dtype=float32)
tf.Tensor(0.08855935, shape=(), dtype=float32)
tf.Tensor

tf.Tensor(0.084592715, shape=(), dtype=float32)
tf.Tensor(0.084569044, shape=(), dtype=float32)
tf.Tensor(0.08454535, shape=(), dtype=float32)
tf.Tensor(0.08452175, shape=(), dtype=float32)
tf.Tensor(0.08449814, shape=(), dtype=float32)
tf.Tensor(0.08447444, shape=(), dtype=float32)
tf.Tensor(0.084450826, shape=(), dtype=float32)
tf.Tensor(0.08442713, shape=(), dtype=float32)
tf.Tensor(0.08440349, shape=(), dtype=float32)
tf.Tensor(0.084379904, shape=(), dtype=float32)
tf.Tensor(0.08435622, shape=(), dtype=float32)
tf.Tensor(0.08433252, shape=(), dtype=float32)
tf.Tensor(0.084308915, shape=(), dtype=float32)
tf.Tensor(0.084285304, shape=(), dtype=float32)
tf.Tensor(0.08426158, shape=(), dtype=float32)
tf.Tensor(0.084237985, shape=(), dtype=float32)
tf.Tensor(0.08421427, shape=(), dtype=float32)
tf.Tensor(0.08419069, shape=(), dtype=float32)
tf.Tensor(0.084167, shape=(), dtype=float32)
tf.Tensor(0.08414332, shape=(), dtype=float32)
tf.Tensor(0.08411968, shape=(), dtype=float32)
tf.Tenso

tf.Tensor(0.080163024, shape=(), dtype=float32)
tf.Tensor(0.08014087, shape=(), dtype=float32)
tf.Tensor(0.080118775, shape=(), dtype=float32)
tf.Tensor(0.08009674, shape=(), dtype=float32)
tf.Tensor(0.08007464, shape=(), dtype=float32)
tf.Tensor(0.080052614, shape=(), dtype=float32)
tf.Tensor(0.08003056, shape=(), dtype=float32)
tf.Tensor(0.08000866, shape=(), dtype=float32)
tf.Tensor(0.07998667, shape=(), dtype=float32)
tf.Tensor(0.0799646, shape=(), dtype=float32)
tf.Tensor(0.07994273, shape=(), dtype=float32)
tf.Tensor(0.07992079, shape=(), dtype=float32)
tf.Tensor(0.0798989, shape=(), dtype=float32)
tf.Tensor(0.07987704, shape=(), dtype=float32)
tf.Tensor(0.07985511, shape=(), dtype=float32)
tf.Tensor(0.079833366, shape=(), dtype=float32)
tf.Tensor(0.07981145, shape=(), dtype=float32)
tf.Tensor(0.07978967, shape=(), dtype=float32)
tf.Tensor(0.079767875, shape=(), dtype=float32)
tf.Tensor(0.07974615, shape=(), dtype=float32)
tf.Tensor(0.07972435, shape=(), dtype=float32)
tf.Tensor(

tf.Tensor(0.076385945, shape=(), dtype=float32)
tf.Tensor(0.0763684, shape=(), dtype=float32)
tf.Tensor(0.0763509, shape=(), dtype=float32)
tf.Tensor(0.07633335, shape=(), dtype=float32)
tf.Tensor(0.07631586, shape=(), dtype=float32)
tf.Tensor(0.07629849, shape=(), dtype=float32)
tf.Tensor(0.07628104, shape=(), dtype=float32)
tf.Tensor(0.07626366, shape=(), dtype=float32)
tf.Tensor(0.07624628, shape=(), dtype=float32)
tf.Tensor(0.07622896, shape=(), dtype=float32)
tf.Tensor(0.07621162, shape=(), dtype=float32)
tf.Tensor(0.0761944, shape=(), dtype=float32)
tf.Tensor(0.07617717, shape=(), dtype=float32)
tf.Tensor(0.076159954, shape=(), dtype=float32)
tf.Tensor(0.0761427, shape=(), dtype=float32)
tf.Tensor(0.07612556, shape=(), dtype=float32)
tf.Tensor(0.076108366, shape=(), dtype=float32)
tf.Tensor(0.076091304, shape=(), dtype=float32)
tf.Tensor(0.07607421, shape=(), dtype=float32)
tf.Tensor(0.07605714, shape=(), dtype=float32)
tf.Tensor(0.07604013, shape=(), dtype=float32)
tf.Tensor(0.0

tf.Tensor(0.073533826, shape=(), dtype=float32)
tf.Tensor(0.073521115, shape=(), dtype=float32)
tf.Tensor(0.07350834, shape=(), dtype=float32)
tf.Tensor(0.073495634, shape=(), dtype=float32)
tf.Tensor(0.07348287, shape=(), dtype=float32)
tf.Tensor(0.07347022, shape=(), dtype=float32)
tf.Tensor(0.073457554, shape=(), dtype=float32)
tf.Tensor(0.073444925, shape=(), dtype=float32)
tf.Tensor(0.07343231, shape=(), dtype=float32)
tf.Tensor(0.073419765, shape=(), dtype=float32)
tf.Tensor(0.07340715, shape=(), dtype=float32)
tf.Tensor(0.073394604, shape=(), dtype=float32)
tf.Tensor(0.073382065, shape=(), dtype=float32)
tf.Tensor(0.07336957, shape=(), dtype=float32)
tf.Tensor(0.07335706, shape=(), dtype=float32)
tf.Tensor(0.07334461, shape=(), dtype=float32)
tf.Tensor(0.07333218, shape=(), dtype=float32)
tf.Tensor(0.073319755, shape=(), dtype=float32)
tf.Tensor(0.073307395, shape=(), dtype=float32)
tf.Tensor(0.07329496, shape=(), dtype=float32)
tf.Tensor(0.07328259, shape=(), dtype=float32)
tf.

In [138]:
ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))


LinearComb
On Training
               precision    recall  f1-score   support

         0.0       0.78      0.99      0.87     19740
         1.0       0.80      0.11      0.20      6308

    accuracy                           0.78     26048
   macro avg       0.79      0.55      0.54     26048
weighted avg       0.78      0.78      0.71     26048

On Testing
               precision    recall  f1-score   support

         0.0       0.79      0.99      0.88      2518
         1.0       0.81      0.09      0.15       738

    accuracy                           0.79      3256
   macro avg       0.80      0.54      0.52      3256
weighted avg       0.79      0.79      0.71      3256



In [ ]:
model2.variables

In [105]:
def taylor_grad(model, data, labels):
    value = model.value(data)
    factor = value/4 - tf.expand_dims(labels, axis=1) + 1/2 #- value**3/48
    w_grad = tf.reduce_mean(factor*data, axis=0)
    b_grad = tf.reduce_mean(factor, axis=0)
    w_grad =  tf.expand_dims(w_grad, axis=1)
    return [w_grad, b_grad]

def taylor_sgd_fn(model, data, labels, last_loss, opt, tol):
    current_loss = model.loss(data, labels)
    grads = taylor_grad(model, data, labels)
#     print(current_loss)
    opt.apply_gradients(zip(grads, model.variables))
    cont = tf.math.abs(last_loss - current_loss) > tol
    return current_loss, cont

def taylor_fit(model, data, labels, max_iter=1000, tol=1e-6):
    opt = tf.keras.optimizers.SGD(learning_rate=0.1)
    last_loss = -np.inf
    for iteration in range(max_iter):
        last_loss, cont = taylor_sgd_fn(model, data, labels, last_loss, opt, tol)
        if not cont:
            break
    print(last_loss)
    print(iteration)
            
def taylor_predict(model, data, ratio):
    return tf.cast(tf.squeeze(model(data) > ratio), tf.int32)

In [141]:
from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, tol=1e-6)

tf.Tensor(0.8896823, shape=(), dtype=float32)
161


In [142]:
ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


TaylorAprx
On Training
               precision    recall  f1-score   support

         0.0       0.80      0.95      0.87     19744
         1.0       0.62      0.27      0.38      6304

    accuracy                           0.78     26048
   macro avg       0.71      0.61      0.62     26048
weighted avg       0.76      0.78      0.75     26048

On Testing
               precision    recall  f1-score   support

         0.0       0.82      0.95      0.88      2493
         1.0       0.63      0.31      0.41       763

    accuracy                           0.80      3256
   macro avg       0.73      0.63      0.65      3256
weighted avg       0.77      0.80      0.77      3256



In [ ]:
model3.variables

In [ ]:
model3(proc.Xtrain)

In [124]:
seed = 2987429
proc = MnistBinaryProcessor(seed)

In [125]:
A_feature = 300
B_feature = 484
A_data = proc.Xtrain[:, :A_feature]
B_data = proc.Xtrain[:, A_feature:]
A_test = proc.Xtest[:, :A_feature]
B_test = proc.Xtest[:, A_feature:]

model = LogReg(proc, 1)
fit(model, A_data, B_data, proc.ytrain)

tf.Tensor(0.28433377, shape=(), dtype=float32)
999


In [126]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, A_data, B_data).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, A_test, B_test).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.88      0.88      0.88     29492
         1.0       0.89      0.89      0.89     30508

    accuracy                           0.88     60000
   macro avg       0.88      0.88      0.88     60000
weighted avg       0.88      0.88      0.88     60000

On Testing
               precision    recall  f1-score   support

         0.0       0.89      0.89      0.89      4926
         1.0       0.89      0.89      0.89      5074

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [127]:
from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

tf.Tensor(0.046283316, shape=(), dtype=float32)
1999


In [128]:
ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))



LinearComb
On Training
               precision    recall  f1-score   support

         0.0       0.87      0.90      0.88     29492
         1.0       0.90      0.87      0.88     30508

    accuracy                           0.88     60000
   macro avg       0.88      0.88      0.88     60000
weighted avg       0.88      0.88      0.88     60000

On Testing
               precision    recall  f1-score   support

         0.0       0.87      0.90      0.89      4926
         1.0       0.90      0.87      0.89      5074

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [129]:
from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, tol=1e-6)

tf.Tensor(0.6160938, shape=(), dtype=float32)
399


In [130]:
ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


TaylorAprx
On Training
               precision    recall  f1-score   support

         0.0       0.89      0.85      0.87     29492
         1.0       0.86      0.90      0.88     30508

    accuracy                           0.88     60000
   macro avg       0.88      0.88      0.88     60000
weighted avg       0.88      0.88      0.88     60000

On Testing
               precision    recall  f1-score   support

         0.0       0.89      0.85      0.87      4926
         1.0       0.86      0.90      0.88      5074

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [20]:
seed = 2987429
proc = BostonHousingPriceProcessor(seed)
model = LogReg(proc, 1)
fit(model, proc.Xtrain, proc.ytrain)

tf.Tensor(0.7750736, shape=(), dtype=float32)
tf.Tensor(0.77087855, shape=(), dtype=float32)
tf.Tensor(0.7669869, shape=(), dtype=float32)
tf.Tensor(0.76337594, shape=(), dtype=float32)
tf.Tensor(0.7600241, shape=(), dtype=float32)
tf.Tensor(0.7569114, shape=(), dtype=float32)
tf.Tensor(0.7540187, shape=(), dtype=float32)
tf.Tensor(0.7513288, shape=(), dtype=float32)
tf.Tensor(0.7488254, shape=(), dtype=float32)
tf.Tensor(0.74649304, shape=(), dtype=float32)
tf.Tensor(0.7443179, shape=(), dtype=float32)
tf.Tensor(0.74228686, shape=(), dtype=float32)
tf.Tensor(0.74038774, shape=(), dtype=float32)
tf.Tensor(0.73860985, shape=(), dtype=float32)
tf.Tensor(0.7369425, shape=(), dtype=float32)
tf.Tensor(0.7353769, shape=(), dtype=float32)
tf.Tensor(0.73390377, shape=(), dtype=float32)
tf.Tensor(0.73251563, shape=(), dtype=float32)
tf.Tensor(0.73120487, shape=(), dtype=float32)
tf.Tensor(0.7299649, shape=(), dtype=float32)
tf.Tensor(0.7287898, shape=(), dtype=float32)
tf.Tensor(0.72767377, sha

tf.Tensor(0.644158, shape=(), dtype=float32)
tf.Tensor(0.64386326, shape=(), dtype=float32)
tf.Tensor(0.6435694, shape=(), dtype=float32)
tf.Tensor(0.6432766, shape=(), dtype=float32)
tf.Tensor(0.6429845, shape=(), dtype=float32)
tf.Tensor(0.6426934, shape=(), dtype=float32)
tf.Tensor(0.6424032, shape=(), dtype=float32)
tf.Tensor(0.64211386, shape=(), dtype=float32)
tf.Tensor(0.64182544, shape=(), dtype=float32)
tf.Tensor(0.64153796, shape=(), dtype=float32)
tf.Tensor(0.64125127, shape=(), dtype=float32)
tf.Tensor(0.6409655, shape=(), dtype=float32)
tf.Tensor(0.6406807, shape=(), dtype=float32)
tf.Tensor(0.64039665, shape=(), dtype=float32)
tf.Tensor(0.6401136, shape=(), dtype=float32)
tf.Tensor(0.6398313, shape=(), dtype=float32)
tf.Tensor(0.6395499, shape=(), dtype=float32)
tf.Tensor(0.6392695, shape=(), dtype=float32)
tf.Tensor(0.6389898, shape=(), dtype=float32)
tf.Tensor(0.638711, shape=(), dtype=float32)
tf.Tensor(0.6384333, shape=(), dtype=float32)
tf.Tensor(0.638156, shape=(), 

tf.Tensor(0.59621775, shape=(), dtype=float32)
tf.Tensor(0.596065, shape=(), dtype=float32)
tf.Tensor(0.5959127, shape=(), dtype=float32)
tf.Tensor(0.5957607, shape=(), dtype=float32)
tf.Tensor(0.59560907, shape=(), dtype=float32)
tf.Tensor(0.59545803, shape=(), dtype=float32)
tf.Tensor(0.5953073, shape=(), dtype=float32)
tf.Tensor(0.595157, shape=(), dtype=float32)
tf.Tensor(0.5950072, shape=(), dtype=float32)
tf.Tensor(0.59485775, shape=(), dtype=float32)
tf.Tensor(0.5947087, shape=(), dtype=float32)
tf.Tensor(0.59456, shape=(), dtype=float32)
tf.Tensor(0.59441185, shape=(), dtype=float32)
tf.Tensor(0.59426403, shape=(), dtype=float32)
tf.Tensor(0.5941165, shape=(), dtype=float32)
tf.Tensor(0.5939694, shape=(), dtype=float32)
tf.Tensor(0.5938229, shape=(), dtype=float32)
tf.Tensor(0.5936766, shape=(), dtype=float32)
tf.Tensor(0.5935307, shape=(), dtype=float32)
tf.Tensor(0.5933853, shape=(), dtype=float32)
tf.Tensor(0.5932403, shape=(), dtype=float32)
tf.Tensor(0.5930958, shape=(), d

tf.Tensor(0.5705128, shape=(), dtype=float32)
tf.Tensor(0.57042634, shape=(), dtype=float32)
tf.Tensor(0.57034016, shape=(), dtype=float32)
tf.Tensor(0.57025427, shape=(), dtype=float32)
tf.Tensor(0.5701685, shape=(), dtype=float32)
tf.Tensor(0.570083, shape=(), dtype=float32)
tf.Tensor(0.5699976, shape=(), dtype=float32)
tf.Tensor(0.5699124, shape=(), dtype=float32)
tf.Tensor(0.56982756, shape=(), dtype=float32)
tf.Tensor(0.5697428, shape=(), dtype=float32)
tf.Tensor(0.56965816, shape=(), dtype=float32)
tf.Tensor(0.5695738, shape=(), dtype=float32)
tf.Tensor(0.5694897, shape=(), dtype=float32)
tf.Tensor(0.5694058, shape=(), dtype=float32)
tf.Tensor(0.56932205, shape=(), dtype=float32)
tf.Tensor(0.56923854, shape=(), dtype=float32)
tf.Tensor(0.5691551, shape=(), dtype=float32)
tf.Tensor(0.56907195, shape=(), dtype=float32)
tf.Tensor(0.56898904, shape=(), dtype=float32)
tf.Tensor(0.5689062, shape=(), dtype=float32)
tf.Tensor(0.5688237, shape=(), dtype=float32)
tf.Tensor(0.5687414, shape

tf.Tensor(0.55544865, shape=(), dtype=float32)
tf.Tensor(0.55539596, shape=(), dtype=float32)
tf.Tensor(0.55534333, shape=(), dtype=float32)
tf.Tensor(0.55529076, shape=(), dtype=float32)
tf.Tensor(0.5552384, shape=(), dtype=float32)
tf.Tensor(0.55518603, shape=(), dtype=float32)
tf.Tensor(0.55513394, shape=(), dtype=float32)
tf.Tensor(0.5550818, shape=(), dtype=float32)
tf.Tensor(0.5550299, shape=(), dtype=float32)
tf.Tensor(0.554978, shape=(), dtype=float32)
tf.Tensor(0.5549262, shape=(), dtype=float32)
tf.Tensor(0.55487454, shape=(), dtype=float32)
tf.Tensor(0.55482304, shape=(), dtype=float32)
tf.Tensor(0.55477154, shape=(), dtype=float32)
tf.Tensor(0.5547202, shape=(), dtype=float32)
tf.Tensor(0.554669, shape=(), dtype=float32)
tf.Tensor(0.5546179, shape=(), dtype=float32)
tf.Tensor(0.5545668, shape=(), dtype=float32)
tf.Tensor(0.55451596, shape=(), dtype=float32)
tf.Tensor(0.55446506, shape=(), dtype=float32)
tf.Tensor(0.55441433, shape=(), dtype=float32)
tf.Tensor(0.5543638, sha

In [21]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, proc.Xtrain).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, proc.Xtest).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.81      0.63      0.71       190
         1.0       0.72      0.87      0.79       214

    accuracy                           0.75       404
   macro avg       0.77      0.75      0.75       404
weighted avg       0.76      0.75      0.75       404

On Testing
               precision    recall  f1-score   support

         0.0       0.77      0.41      0.53        56
         1.0       0.54      0.85      0.66        46

    accuracy                           0.61       102
   macro avg       0.65      0.63      0.60       102
weighted avg       0.67      0.61      0.59       102



In [38]:
A_feature = 6
A_data = proc.Xtrain[:, :A_feature]
B_data = proc.Xtrain[:, A_feature:]
A_test = proc.Xtest[:, :A_feature]
B_test = proc.Xtest[:, A_feature:]

from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

tf.Tensor(0.12843628, shape=(), dtype=float32)
tf.Tensor(0.12774996, shape=(), dtype=float32)
tf.Tensor(0.12713656, shape=(), dtype=float32)
tf.Tensor(0.12658766, shape=(), dtype=float32)
tf.Tensor(0.12609588, shape=(), dtype=float32)
tf.Tensor(0.12565482, shape=(), dtype=float32)
tf.Tensor(0.12525871, shape=(), dtype=float32)
tf.Tensor(0.124902554, shape=(), dtype=float32)
tf.Tensor(0.12458195, shape=(), dtype=float32)
tf.Tensor(0.124293014, shape=(), dtype=float32)
tf.Tensor(0.12403222, shape=(), dtype=float32)
tf.Tensor(0.1237966, shape=(), dtype=float32)
tf.Tensor(0.12358338, shape=(), dtype=float32)
tf.Tensor(0.12339022, shape=(), dtype=float32)
tf.Tensor(0.123214945, shape=(), dtype=float32)
tf.Tensor(0.123055674, shape=(), dtype=float32)
tf.Tensor(0.122910745, shape=(), dtype=float32)
tf.Tensor(0.12277862, shape=(), dtype=float32)
tf.Tensor(0.122658, shape=(), dtype=float32)
tf.Tensor(0.12254766, shape=(), dtype=float32)
tf.Tensor(0.12244657, shape=(), dtype=float32)
tf.Tensor(0

tf.Tensor(0.118054494, shape=(), dtype=float32)
tf.Tensor(0.11803655, shape=(), dtype=float32)
tf.Tensor(0.11801861, shape=(), dtype=float32)
tf.Tensor(0.11800064, shape=(), dtype=float32)
tf.Tensor(0.11798269, shape=(), dtype=float32)
tf.Tensor(0.117964745, shape=(), dtype=float32)
tf.Tensor(0.117946796, shape=(), dtype=float32)
tf.Tensor(0.117928855, shape=(), dtype=float32)
tf.Tensor(0.11791091, shape=(), dtype=float32)
tf.Tensor(0.117892936, shape=(), dtype=float32)
tf.Tensor(0.11787498, shape=(), dtype=float32)
tf.Tensor(0.11785704, shape=(), dtype=float32)
tf.Tensor(0.117839046, shape=(), dtype=float32)
tf.Tensor(0.1178211, shape=(), dtype=float32)
tf.Tensor(0.11780313, shape=(), dtype=float32)
tf.Tensor(0.11778519, shape=(), dtype=float32)
tf.Tensor(0.117767215, shape=(), dtype=float32)
tf.Tensor(0.11774926, shape=(), dtype=float32)
tf.Tensor(0.11773128, shape=(), dtype=float32)
tf.Tensor(0.11771333, shape=(), dtype=float32)
tf.Tensor(0.11769536, shape=(), dtype=float32)
tf.Tens

tf.Tensor(0.114064135, shape=(), dtype=float32)
tf.Tensor(0.11404578, shape=(), dtype=float32)
tf.Tensor(0.11402742, shape=(), dtype=float32)
tf.Tensor(0.11400907, shape=(), dtype=float32)
tf.Tensor(0.11399069, shape=(), dtype=float32)
tf.Tensor(0.11397234, shape=(), dtype=float32)
tf.Tensor(0.11395397, shape=(), dtype=float32)
tf.Tensor(0.113935605, shape=(), dtype=float32)
tf.Tensor(0.11391724, shape=(), dtype=float32)
tf.Tensor(0.113898866, shape=(), dtype=float32)
tf.Tensor(0.1138805, shape=(), dtype=float32)
tf.Tensor(0.113862135, shape=(), dtype=float32)
tf.Tensor(0.11384375, shape=(), dtype=float32)
tf.Tensor(0.11382537, shape=(), dtype=float32)
tf.Tensor(0.11380701, shape=(), dtype=float32)
tf.Tensor(0.113788605, shape=(), dtype=float32)
tf.Tensor(0.11377023, shape=(), dtype=float32)
tf.Tensor(0.11375186, shape=(), dtype=float32)
tf.Tensor(0.11373348, shape=(), dtype=float32)
tf.Tensor(0.113715075, shape=(), dtype=float32)
tf.Tensor(0.11369671, shape=(), dtype=float32)
tf.Tenso

tf.Tensor(0.10998253, shape=(), dtype=float32)
tf.Tensor(0.10996429, shape=(), dtype=float32)
tf.Tensor(0.10994604, shape=(), dtype=float32)
tf.Tensor(0.10992782, shape=(), dtype=float32)
tf.Tensor(0.10990961, shape=(), dtype=float32)
tf.Tensor(0.109891385, shape=(), dtype=float32)
tf.Tensor(0.10987316, shape=(), dtype=float32)
tf.Tensor(0.109854944, shape=(), dtype=float32)
tf.Tensor(0.10983674, shape=(), dtype=float32)
tf.Tensor(0.1098185, shape=(), dtype=float32)
tf.Tensor(0.10980031, shape=(), dtype=float32)
tf.Tensor(0.109782115, shape=(), dtype=float32)
tf.Tensor(0.10976393, shape=(), dtype=float32)
tf.Tensor(0.10974574, shape=(), dtype=float32)
tf.Tensor(0.10972754, shape=(), dtype=float32)
tf.Tensor(0.10970935, shape=(), dtype=float32)
tf.Tensor(0.109691165, shape=(), dtype=float32)
tf.Tensor(0.10967298, shape=(), dtype=float32)
tf.Tensor(0.109654814, shape=(), dtype=float32)
tf.Tensor(0.10963663, shape=(), dtype=float32)
tf.Tensor(0.10961846, shape=(), dtype=float32)
tf.Tensor

tf.Tensor(0.10602434, shape=(), dtype=float32)
tf.Tensor(0.10600746, shape=(), dtype=float32)
tf.Tensor(0.10599055, shape=(), dtype=float32)
tf.Tensor(0.10597366, shape=(), dtype=float32)
tf.Tensor(0.10595681, shape=(), dtype=float32)
tf.Tensor(0.10593993, shape=(), dtype=float32)
tf.Tensor(0.10592308, shape=(), dtype=float32)
tf.Tensor(0.105906226, shape=(), dtype=float32)
tf.Tensor(0.10588939, shape=(), dtype=float32)
tf.Tensor(0.10587256, shape=(), dtype=float32)
tf.Tensor(0.10585572, shape=(), dtype=float32)
tf.Tensor(0.105838925, shape=(), dtype=float32)
tf.Tensor(0.1058221, shape=(), dtype=float32)
tf.Tensor(0.10580532, shape=(), dtype=float32)
tf.Tensor(0.10578852, shape=(), dtype=float32)
tf.Tensor(0.105771735, shape=(), dtype=float32)
tf.Tensor(0.105754964, shape=(), dtype=float32)
tf.Tensor(0.105738215, shape=(), dtype=float32)
tf.Tensor(0.10572147, shape=(), dtype=float32)
tf.Tensor(0.10570471, shape=(), dtype=float32)
tf.Tensor(0.10568798, shape=(), dtype=float32)
tf.Tensor

In [39]:
ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))



LinearComb
On Training
               precision    recall  f1-score   support

         0.0       0.82      0.52      0.64       190
         1.0       0.68      0.90      0.78       214

    accuracy                           0.72       404
   macro avg       0.75      0.71      0.71       404
weighted avg       0.75      0.72      0.71       404

On Testing
               precision    recall  f1-score   support

         0.0       0.78      0.38      0.51        56
         1.0       0.53      0.87      0.66        46

    accuracy                           0.60       102
   macro avg       0.66      0.62      0.58       102
weighted avg       0.67      0.60      0.58       102



In [40]:
from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, max_iter=1000)

tf.Tensor(0.70447767, shape=(), dtype=float32)
tf.Tensor(0.703289, shape=(), dtype=float32)
tf.Tensor(0.7021303, shape=(), dtype=float32)
tf.Tensor(0.70100087, shape=(), dtype=float32)
tf.Tensor(0.6998997, shape=(), dtype=float32)
tf.Tensor(0.6988259, shape=(), dtype=float32)
tf.Tensor(0.69777876, shape=(), dtype=float32)
tf.Tensor(0.6967575, shape=(), dtype=float32)
tf.Tensor(0.6957613, shape=(), dtype=float32)
tf.Tensor(0.6947895, shape=(), dtype=float32)
tf.Tensor(0.69384134, shape=(), dtype=float32)
tf.Tensor(0.6929161, shape=(), dtype=float32)
tf.Tensor(0.6920132, shape=(), dtype=float32)
tf.Tensor(0.69113195, shape=(), dtype=float32)
tf.Tensor(0.6902718, shape=(), dtype=float32)
tf.Tensor(0.689432, shape=(), dtype=float32)
tf.Tensor(0.6886121, shape=(), dtype=float32)
tf.Tensor(0.68781143, shape=(), dtype=float32)
tf.Tensor(0.6870296, shape=(), dtype=float32)
tf.Tensor(0.68626595, shape=(), dtype=float32)
tf.Tensor(0.68552, shape=(), dtype=float32)
tf.Tensor(0.6847912, shape=(), 

tf.Tensor(0.6419553, shape=(), dtype=float32)
tf.Tensor(0.641899, shape=(), dtype=float32)
tf.Tensor(0.6418428, shape=(), dtype=float32)
tf.Tensor(0.6417868, shape=(), dtype=float32)
tf.Tensor(0.641731, shape=(), dtype=float32)
tf.Tensor(0.6416754, shape=(), dtype=float32)
tf.Tensor(0.64162, shape=(), dtype=float32)
tf.Tensor(0.6415648, shape=(), dtype=float32)
tf.Tensor(0.6415097, shape=(), dtype=float32)
tf.Tensor(0.6414549, shape=(), dtype=float32)
tf.Tensor(0.64140016, shape=(), dtype=float32)
tf.Tensor(0.6413457, shape=(), dtype=float32)
tf.Tensor(0.6412914, shape=(), dtype=float32)
tf.Tensor(0.6412372, shape=(), dtype=float32)
tf.Tensor(0.64118326, shape=(), dtype=float32)
tf.Tensor(0.6411295, shape=(), dtype=float32)
tf.Tensor(0.64107585, shape=(), dtype=float32)
tf.Tensor(0.64102244, shape=(), dtype=float32)
tf.Tensor(0.6409692, shape=(), dtype=float32)
tf.Tensor(0.6409161, shape=(), dtype=float32)
tf.Tensor(0.64086324, shape=(), dtype=float32)
tf.Tensor(0.6408105, shape=(), dt

tf.Tensor(0.6319451, shape=(), dtype=float32)
tf.Tensor(0.63192165, shape=(), dtype=float32)
tf.Tensor(0.63189834, shape=(), dtype=float32)
tf.Tensor(0.6318751, shape=(), dtype=float32)
tf.Tensor(0.631852, shape=(), dtype=float32)
tf.Tensor(0.6318289, shape=(), dtype=float32)
tf.Tensor(0.63180596, shape=(), dtype=float32)
tf.Tensor(0.63178307, shape=(), dtype=float32)
tf.Tensor(0.63176024, shape=(), dtype=float32)
tf.Tensor(0.63173753, shape=(), dtype=float32)
tf.Tensor(0.63171494, shape=(), dtype=float32)
tf.Tensor(0.63169235, shape=(), dtype=float32)
tf.Tensor(0.63166994, shape=(), dtype=float32)
tf.Tensor(0.6316475, shape=(), dtype=float32)
tf.Tensor(0.6316253, shape=(), dtype=float32)
tf.Tensor(0.63160306, shape=(), dtype=float32)
tf.Tensor(0.63158095, shape=(), dtype=float32)
tf.Tensor(0.6315589, shape=(), dtype=float32)
tf.Tensor(0.631537, shape=(), dtype=float32)
tf.Tensor(0.63151515, shape=(), dtype=float32)
tf.Tensor(0.63149333, shape=(), dtype=float32)
tf.Tensor(0.63147163, s

In [41]:
ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


TaylorAprx
On Training
               precision    recall  f1-score   support

         0.0       0.82      0.54      0.65       190
         1.0       0.69      0.89      0.78       214

    accuracy                           0.73       404
   macro avg       0.75      0.72      0.71       404
weighted avg       0.75      0.73      0.72       404

On Testing
               precision    recall  f1-score   support

         0.0       0.79      0.39      0.52        56
         1.0       0.54      0.87      0.67        46

    accuracy                           0.61       102
   macro avg       0.66      0.63      0.60       102
weighted avg       0.68      0.61      0.59       102



In [77]:
seed = 2987429
proc = BreastCancerProcessor(seed)
model = LogReg(proc, 1)
fit(model, proc.Xtrain, proc.ytrain)

A_feature = 14
A_data = proc.Xtrain[:, :A_feature]
B_data = proc.Xtrain[:, A_feature:]
A_test = proc.Xtest[:, :A_feature]
B_test = proc.Xtest[:, A_feature:]

from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, max_iter=1000)

tf.Tensor(0.6121137, shape=(), dtype=float32)
999
tf.Tensor(0.115053356, shape=(), dtype=float32)
999
tf.Tensor(0.5606389, shape=(), dtype=float32)
203


In [78]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, proc.Xtrain).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, proc.Xtest).numpy()))


ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))

ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       1.00      0.01      0.01       170
         1.0       0.63      1.00      0.77       285

    accuracy                           0.63       455
   macro avg       0.81      0.50      0.39       455
weighted avg       0.77      0.63      0.49       455

On Testing
               precision    recall  f1-score   support

         0.0       1.00      0.02      0.05        42
         1.0       0.64      1.00      0.78        72

    accuracy                           0.64       114
   macro avg       0.82      0.51      0.41       114
weighted avg       0.77      0.64      0.51       114

LinearComb
On Training
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       170
         1.0       0.63      1.00      0.77       285

    accuracy                           0.63       455
   macro avg       0.31      0.50      0.39       455
weighted avg      

/home/yejia/.local/share/virtualenvs/Rain-Experiments-jsPNbhrB/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [68]:
proc.Xtrain

<tf.Tensor: shape=(455, 30), dtype=float32, numpy=
array([[1.85450185e-02, 2.55055726e-02, 1.20748311e-01, ...,
        1.36117480e-04, 3.54938820e-04, 1.71627820e-04],
       [1.56324115e-02, 2.24538259e-02, 1.02185301e-01, ...,
        1.73871958e-04, 3.99151671e-04, 1.27654392e-04],
       [1.66033451e-02, 4.05543819e-02, 1.04762107e-01, ...,
        0.00000000e+00, 3.25977657e-04, 9.12439791e-05],
       ...,
       [1.48008848e-02, 2.20767017e-02, 9.62472931e-02, ...,
        1.20472323e-04, 3.57737503e-04, 1.03855447e-04],
       [1.69124864e-02, 2.18624827e-02, 1.08391166e-01, ...,
        6.62062012e-05, 3.41274717e-04, 8.67074341e-05],
       [1.69538353e-02, 2.20643207e-02, 1.09848410e-01, ...,
        1.13715076e-04, 4.26549814e-04, 1.01912286e-04]], dtype=float32)>

In [69]:
proc.ytrain

<tf.Tensor: shape=(455,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
       0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1

In [70]:
model(proc.Xtrain)

<tf.Tensor: shape=(455, 1), dtype=float32, numpy=
array([[0.6577544 ],
       [0.6266426 ],
       [0.6462515 ],
       [0.562672  ],
       [0.616267  ],
       [0.61413753],
       [0.64967597],
       [0.64668864],
       [0.6176642 ],
       [0.58659804],
       [0.62844   ],
       [0.6463012 ],
       [0.6461513 ],
       [0.6525389 ],
       [0.6707817 ],
       [0.63031936],
       [0.6519958 ],
       [0.6472025 ],
       [0.5586817 ],
       [0.6055135 ],
       [0.62675524],
       [0.6434339 ],
       [0.5101375 ],
       [0.6467135 ],
       [0.60359365],
       [0.64426064],
       [0.64742607],
       [0.6534106 ],
       [0.64938045],
       [0.6480498 ],
       [0.62887484],
       [0.633792  ],
       [0.6642    ],
       [0.6488075 ],
       [0.65638566],
       [0.5926576 ],
       [0.64952695],
       [0.6432003 ],
       [0.5932031 ],
       [0.6306252 ],
       [0.5451064 ],
       [0.6096225 ],
       [0.63657665],
       [0.6759114 ],
       [0.6088914 ],
     

In [71]:
model2(A_data, B_data)

<tf.Tensor: shape=(455, 1), dtype=float32, numpy=
array([[0.6324751 ],
       [0.62842613],
       [0.6282568 ],
       [0.6222787 ],
       [0.62521994],
       [0.62620187],
       [0.6300117 ],
       [0.6311758 ],
       [0.6277715 ],
       [0.6234525 ],
       [0.6275691 ],
       [0.62859917],
       [0.6289704 ],
       [0.6339529 ],
       [0.63220924],
       [0.6272192 ],
       [0.62952983],
       [0.63090897],
       [0.62118924],
       [0.6262162 ],
       [0.62651837],
       [0.63094425],
       [0.61699206],
       [0.6348646 ],
       [0.62683356],
       [0.62817   ],
       [0.6324533 ],
       [0.6279769 ],
       [0.6305104 ],
       [0.6311127 ],
       [0.6276242 ],
       [0.62901664],
       [0.63208747],
       [0.6283194 ],
       [0.6318568 ],
       [0.6243706 ],
       [0.6313392 ],
       [0.6283766 ],
       [0.62405324],
       [0.6279466 ],
       [0.6206506 ],
       [0.6261497 ],
       [0.6273786 ],
       [0.6325382 ],
       [0.6258665 ],
     

In [116]:
seed = 2987429
proc = DiabetesProcessor(seed)
A_feature = 5
A_data = proc.Xtrain[:, :A_feature]
B_data = proc.Xtrain[:, A_feature:]
A_test = proc.Xtest[:, :A_feature]
B_test = proc.Xtest[:, A_feature:]

model = LogReg(proc, 1)
fit(model, A_data, B_data, proc.ytrain)

from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, max_iter=1000)

tf.Tensor(0.49575517, shape=(), dtype=float32)
993
tf.Tensor(0.08675196, shape=(), dtype=float32)
1999
tf.Tensor(0.6365558, shape=(), dtype=float32)
999


In [118]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, A_data, B_data).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, A_test, B_test).numpy()))


ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))

ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.75      0.72      0.74       164
         1.0       0.77      0.79      0.78       189

    accuracy                           0.76       353
   macro avg       0.76      0.76      0.76       353
weighted avg       0.76      0.76      0.76       353

On Testing
               precision    recall  f1-score   support

         0.0       0.86      0.77      0.81        57
         1.0       0.66      0.78      0.71        32

    accuracy                           0.78        89
   macro avg       0.76      0.78      0.76        89
weighted avg       0.79      0.78      0.78        89

LinearComb
On Training
               precision    recall  f1-score   support

         0.0       0.73      0.69      0.71       164
         1.0       0.74      0.78      0.76       189

    accuracy                           0.74       353
   macro avg       0.74      0.73      0.73       353
weighted avg      

In [114]:
seed = 2987429
proc = IrisProcessor(seed)

A_feature = 1
A_data = proc.Xtrain[:, :A_feature]
B_data = proc.Xtrain[:, A_feature:]
A_test = proc.Xtest[:, :A_feature]
B_test = proc.Xtest[:, A_feature:]

model = LogReg(proc, 1)
fit(model, A_data, B_data, proc.ytrain)

from mlsql.models.linear_comb import LinearComb
model2 = LinearComb(proc, 1)
new_fit(model2, A_data, B_data, proc.ytrain)

from mlsql.models.linear_comb import TaylorAprx
model3 = TaylorAprx(proc, 1)
taylor_fit(model3, proc.Xtrain, proc.ytrain, tol=1e-8)

tf.Tensor(0.23064372, shape=(), dtype=float32)
999
tf.Tensor(0.030700581, shape=(), dtype=float32)
1999
tf.Tensor(0.38212588, shape=(), dtype=float32)
914


In [115]:
print("LogReg")
print("On Training\n", classification_report(proc.ytrain.numpy(), predict(model, A_data, B_data).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), predict(model, A_test, B_test).numpy()))


ratio=0.5
print("LinearComb")
print("On Training\n", classification_report(proc.ytrain.numpy(), new_predict(model2, A_data, B_data, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), new_predict(model2, A_test, B_test, ratio).numpy()))

ratio=0.5
print("TaylorAprx")
print("On Training\n", classification_report(proc.ytrain.numpy(), taylor_predict(model3, proc.Xtrain, ratio).numpy()))
print("On Testing\n", classification_report(proc.ytest.numpy(), taylor_predict(model3, proc.Xtest, ratio).numpy()))


LogReg
On Training
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        37
         1.0       1.00      1.00      1.00        83

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

On Testing
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        13
         1.0       1.00      1.00      1.00        17

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

LinearComb
On Training
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        37
         1.0       1.00      1.00      1.00        83

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg      